# Ingestão de Dados - SQL Server

Este notebook realiza a ingestão de dados de um banco de dados SQL Server para o MinIO usando DeltaLake.

## Configuração

Configure as variáveis abaixo antes de executar:

In [ ]:
# Importar configurações base
%run ../00_configuracao_inicial.ipynb

In [ ]:
# ============================================
# CONFIGURAÇÕES DE CONEXÃO SQL SERVER
# ============================================
import os

# Configurações de conexão SQL Server
SQLSERVER_HOST = os.getenv('SQLSERVER_HOST', 'localhost')
SQLSERVER_PORT = os.getenv('SQLSERVER_PORT', '1433')
SQLSERVER_DATABASE = os.getenv('SQLSERVER_DATABASE', 'master')
SQLSERVER_USER = os.getenv('SQLSERVER_USER', 'sa')
SQLSERVER_PASSWORD = os.getenv('SQLSERVER_PASSWORD', 'senha')

# Configurações de leitura
SQLSERVER_SCHEMA = os.getenv('SQLSERVER_SCHEMA', 'dbo')
SQLSERVER_TABLE = os.getenv('SQLSERVER_TABLE', 'nome_tabela')

# Configurações de destino no MinIO
DESTINO_BRONZE = f"{PATH_BRONZE}/sqlserver/{SQLSERVER_DATABASE.lower()}/{SQLSERVER_SCHEMA.lower()}/{SQLSERVER_TABLE.lower()}"

print("Configurações SQL Server:")
print(f"Host: {SQLSERVER_HOST}")
print(f"Port: {SQLSERVER_PORT}")
print(f"Database: {SQLSERVER_DATABASE}")
print(f"Schema: {SQLSERVER_SCHEMA}")
print(f"Table: {SQLSERVER_TABLE}")
print(f"Destino: {DESTINO_BRONZE}")

In [ ]:
# Instalar driver SQL Server (executar apenas uma vez)
# !pip install pymssql

# Adicionar driver JDBC SQL Server ao Spark
# Baixar mssql-jdbc-XX.X.X.jre8.jar e colocar no classpath do Spark
# Ou usar: spark.jars.packages com coordenadas Maven: com.microsoft.sqlserver:mssql-jdbc:XX.X.X

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# URL de conexão JDBC SQL Server
# Formato: jdbc:sqlserver://[serverName[\instanceName][:portNumber]][;property=value[;property=value]]
jdbc_url = f"jdbc:sqlserver://{SQLSERVER_HOST}:{SQLSERVER_PORT};databaseName={SQLSERVER_DATABASE};encrypt=true;trustServerCertificate=true"

# Propriedades de conexão
connection_properties = {
    "user": SQLSERVER_USER,
    "password": SQLSERVER_PASSWORD,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

print(f"JDBC URL: {jdbc_url}")

In [ ]:
# Função para ler dados do SQL Server
def ler_sqlserver_table(table_name, schema="dbo", query=None, partition_column=None, num_partitions=None, lower_bound=None, upper_bound=None):
    """
    Lê dados de uma tabela SQL Server
    
    Args:
        table_name: Nome da tabela
        schema: Schema (padrão: dbo)
        query: Query SQL customizada (opcional, substitui table_name)
        partition_column: Coluna para particionamento paralelo (opcional)
        num_partitions: Número de partições (opcional)
        lower_bound: Valor mínimo para particionamento (opcional)
        upper_bound: Valor máximo para particionamento (opcional)
    
    Returns:
        DataFrame do Spark
    """
    if query:
        # Usar query customizada (subquery)
        table_or_query = f"({query}) sqlserver_table"
    elif schema:
        table_or_query = f"{schema}.{table_name}"
    else:
        table_or_query = table_name
    
    reader = spark.read.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", table_or_query) \
        .option("user", SQLSERVER_USER) \
        .option("password", SQLSERVER_PASSWORD) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
    
    # Adicionar opções de particionamento se fornecidas
    if partition_column and num_partitions:
        reader = reader.option("partitionColumn", partition_column) \
                      .option("numPartitions", num_partitions)
        if lower_bound is not None and upper_bound is not None:
            reader = reader.option("lowerBound", lower_bound) \
                          .option("upperBound", upper_bound)
    
    df = reader.load()
    
    return df

In [ ]:
# Exemplo 1: Leitura simples de tabela
print("Exemplo 1: Leitura simples")
df_sqlserver = ler_sqlserver_table(
    table_name=SQLSERVER_TABLE,
    schema=SQLSERVER_SCHEMA
)

print(f"Total de registros: {df_sqlserver.count()}")
df_sqlserver.printSchema()
df_sqlserver.show(5, truncate=False)

In [ ]:
# Exemplo 2: Leitura com query customizada
print("Exemplo 2: Leitura com query customizada")
query_customizada = f"""
    SELECT 
        coluna1,
        coluna2,
        coluna3,
        UpdatedAt
    FROM {SQLSERVER_SCHEMA}.{SQLSERVER_TABLE}
    WHERE UpdatedAt >= DATEADD(day, -30, GETDATE())
    ORDER BY UpdatedAt DESC
"""

# df_sqlserver_query = ler_sqlserver_table(query=query_customizada)
# df_sqlserver_query.show(5)

In [ ]:
# Exemplo 3: Leitura com particionamento paralelo (para tabelas grandes)
print("Exemplo 3: Leitura com particionamento")
# df_sqlserver_partitioned = ler_sqlserver_table(
#     table_name=SQLSERVER_TABLE,
#     schema=SQLSERVER_SCHEMA,
#     partition_column="Id",  # Coluna numérica para particionamento
#     num_partitions=10,
#     lower_bound=1,
#     upper_bound=1000000
# )
# df_sqlserver_partitioned.show(5)

In [ ]:
# Adicionar metadados de ingestão
df_ingestao = df_sqlserver \
    .withColumn("fonte", lit("SQLSERVER")) \
    .withColumn("database_origem", lit(SQLSERVER_DATABASE)) \
    .withColumn("schema_origem", lit(SQLSERVER_SCHEMA)) \
    .withColumn("tabela_origem", lit(SQLSERVER_TABLE)) \
    .withColumn("ingestao_em", current_timestamp()) \
    .withColumn("particao_data", date_format(current_date(), "yyyy-MM-dd"))

print("Metadados adicionados:")
df_ingestao.select("fonte", "database_origem", "schema_origem", "tabela_origem", "ingestao_em").show(1, truncate=False)

In [ ]:
# Salvar no MinIO como Delta Table
print(f"Salvando dados em: {DESTINO_BRONZE}")

# save_delta_table(
#     df_ingestao,
#     DESTINO_BRONZE,
#     mode="overwrite",  # ou "append" para incrementais
#     partition_by=["particao_data"]  # Particionar por data
# )

print("Ingestão concluída com sucesso!")

In [ ]:
# Verificar dados salvos
# df_verificacao = read_delta_table(DESTINO_BRONZE)
# print(f"Registros salvos: {df_verificacao.count()}")
# df_verificacao.show(5)

## Ingestão Incremental

Para ingestões incrementais baseadas em timestamp ou ID:

In [ ]:
# Função para ingestão incremental
def ingestao_incremental_sqlserver(table_name, schema, coluna_timestamp="UpdatedAt", ultima_execucao=None):
    """
    Realiza ingestão incremental de dados SQL Server
    
    Args:
        table_name: Nome da tabela
        schema: Schema
        coluna_timestamp: Nome da coluna de timestamp para filtro
        ultima_execucao: Timestamp da última execução (formato: 'YYYY-MM-DD HH24:MI:SS')
    """
    if ultima_execucao:
        query = f"""
            SELECT * FROM {schema}.{table_name}
            WHERE {coluna_timestamp} > '{ultima_execucao}'
            ORDER BY {coluna_timestamp}
        """
    else:
        # Primeira execução: pegar últimos 7 dias
        query = f"""
            SELECT * FROM {schema}.{table_name}
            WHERE {coluna_timestamp} >= DATEADD(day, -7, GETDATE())
            ORDER BY {coluna_timestamp}
        """
    
    df_incremental = ler_sqlserver_table(query=query)
    
    return df_incremental

# Exemplo de uso
# df_incremental = ingestao_incremental_sqlserver(
#     table_name=SQLSERVER_TABLE,
#     schema=SQLSERVER_SCHEMA,
#     ultima_execucao="2024-01-01 00:00:00"
# )
# 
# # Salvar em modo append
# save_delta_table(df_incremental, DESTINO_BRONZE, mode="append", partition_by=["particao_data"])